In [1]:
import os
import sys
import json
from anthropic import Anthropic, AnthropicError

notebook_path = os.getcwd()
project_path = os.path.abspath(os.path.join(notebook_path, '..','..','..'))
print(f"Project path: {project_path}")
# Add the project path to sys.path
sys.path.append(project_path)

Project path: c:\Users\dalej\Documents\_Coding\DragonRegen


## Testing Action Generator

In [2]:
s_test_json = """{"users": {     "purpose": "Stores information about registered users/customers",     "fields": {       "user_id": "unique ID, primary key for user identification",       "username": "unique username for login",       "email": "unique email address for user contact",       "password_hash": "securely stored password hash",       "first_name": "user's first name",       "last_name": "user's last name",       "phone_number": "contact phone number",       "created_at": "timestamp when user account was created",       "updated_at": "timestamp when user account was last updated"     }   },   "addresses": {     "purpose": "Stores shipping and billing addresses for users",     "fields": {       "address_id": "unique ID, primary key for address identification",       "user_id": "foreign key referencing users table",       "address_type": "type of address (shipping/billing)",       "street_address": "street name and number",       "city": "city name",       "state": "state or province",       "postal_code": "zip or postal code",       "country": "country name",       "is_default": "boolean indicating if this is the default address for the user"     }   },   "products": {     "purpose": "Stores product information available for sale",     "fields": {       "product_id": "unique ID, primary key for product identification",       "name": "product name",       "description": "detailed product description",       "price": "current product price",       "category_id": "foreign key referencing categories table",       "inventory_count": "current available inventory",       "image_url": "URL to product image",       "created_at": "timestamp when product was added",       "updated_at": "timestamp when product was last updated"     }   },   "categories": {     "purpose": "Stores product categories for organization",     "fields": {       "category_id": "unique ID, primary key for category identification",       "name": "category name",       "description": "category description",       "parent_category_id": "self-referencing foreign key for hierarchical categories"     }   },   "orders": {     "purpose": "Stores customer orders",     "fields": {       "order_id": "unique ID, primary key for order identification",       "user_id": "foreign key referencing users table",       "order_date": "date and time when order was placed",       "total_amount": "total order amount",       "shipping_address_id": "foreign key referencing addresses table",       "billing_address_id": "foreign key referencing addresses table",       "status": "current order status (pending, shipped, delivered, etc.)",       "payment_method": "method used for payment"     }   },   "order_items": {     "purpose": "Stores individual items within an order",     "columns": {       "order_item_id": "unique ID, primary key for order item identification",       "order_id": "foreign key referencing orders table",       "product_id": "foreign key referencing products table",       "quantity": "quantity of product ordered",       "unit_price": "price of product at time of purchase",       "subtotal": "total price for this line item (quantity * unit_price)"     }   },   "payments": {     "purpose": "Stores payment information for orders",     "fields": {       "payment_id": "unique ID, primary key for payment identification",       "order_id": "foreign key referencing orders table",       "payment_date": "date and time of payment",       "payment_method": "method of payment (credit card, PayPal, etc.)",       "amount": "payment amount",       "status": "payment status (pending, completed, failed, etc.)",       "transaction_id": "external payment processor transaction ID"     }   },   "cart_items": {     "purpose": "Stores items in user shopping carts",     "fields": {       "cart_item_id": "unique ID, primary key for cart item identification",       "user_id": "foreign key referencing users table",       "product_id": "foreign key referencing products table",       "quantity": "quantity of product in cart",       "added_at": "timestamp when item was added to cart"     }   },   "reviews": {     "purpose": "Stores product reviews from users",     "columns": {       "review_id": "unique ID, primary key for review identification",       "product_id": "foreign key referencing products table",       "user_id": "foreign key referencing users table",       "rating": "numerical rating (e.g., 1-5 stars)",       "comment": "text review content",       "created_at": "timestamp when review was created"     }   } }"""
test_json = json.loads(s_test_json)
for key in test_json.keys():
    print(key) 

users
addresses
products
categories
orders
order_items
payments
cart_items
reviews


In [3]:
schema_json = {key: val for key, val in test_json.items() if key in ['addresses','products', 'categories']}
print(schema_json.keys())
print(schema_json)

dict_keys(['addresses', 'products', 'categories'])
{'addresses': {'purpose': 'Stores shipping and billing addresses for users', 'fields': {'address_id': 'unique ID, primary key for address identification', 'user_id': 'foreign key referencing users table', 'address_type': 'type of address (shipping/billing)', 'street_address': 'street name and number', 'city': 'city name', 'state': 'state or province', 'postal_code': 'zip or postal code', 'country': 'country name', 'is_default': 'boolean indicating if this is the default address for the user'}}, 'products': {'purpose': 'Stores product information available for sale', 'fields': {'product_id': 'unique ID, primary key for product identification', 'name': 'product name', 'description': 'detailed product description', 'price': 'current product price', 'category_id': 'foreign key referencing categories table', 'inventory_count': 'current available inventory', 'image_url': 'URL to product image', 'created_at': 'timestamp when product was added

In [ ]:
from src.AIGuardian.Tasks.SchemaRefiner import SchemaRefiner

test_schema_refiner = SchemaRefiner(schema_json)
print(test_schema_refiner)


In [7]:
test_schema_refiner.run(user_prompt="")

create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': 'Please take the following <column_information> to choose what type of column a column most likely is.\n<column_information>\nTable Purpose: Stores shipping and billing addresses for users\nColumn Info: "address_id": "unique ID, primary key for address identification"\n</column_information>\n\nChoose one column_type from this list <choices> below, structure as column_type: description of what qualifies a column as that type.\n<choices>\nColBasic: A back stop column for your basic data types with no other identifying attributes.\nStringBasic: This is a basic string column generator that can be used to generate a column of strings with a specified length and format. It is not intended for free text or complex string generation.\nCategorical: a column that represents a category or type, often with a limited set of 

AttributeError: 'dict' object has no attribute 'append'

## Test Table Column Action Generator

In [4]:
table_json = test_json['users']
table_json['table_name'] = 'users'
print(table_json)

{'purpose': 'Stores information about registered users/customers', 'fields': {'user_id': 'unique ID, primary key for user identification', 'username': 'unique username for login', 'email': 'unique email address for user contact', 'password_hash': 'securely stored password hash', 'first_name': "user's first name", 'last_name': "user's last name", 'phone_number': 'contact phone number', 'created_at': 'timestamp when user account was created', 'updated_at': 'timestamp when user account was last updated'}, 'table_name': 'users'}


In [8]:
from src.AIGuardian.Tasks.ColumnRefiner import ColumnRefiner
from src.MetaFort.SysLogs.KafkaEngine import KafkaEngine

test_col_refiner = ColumnRefiner(input_params=table_json)
test_col_refiner.db_engine = KafkaEngine.default_builder()
print(test_col_refiner)

In [ ]:
await test_col_refiner.run(user_prompt="")


In [6]:
print(json.dumps(test_col_refiner.output_params, indent=2))

{
  "users": {
    "purpose": "Stores information about registered users/customers",
    "fields": [
      {
        "name": "user_id",
        "type": "Integer",
        "nullable": false,
        "metadata": {
          "description": "Unique ID, primary key for user identification in the registered users/customers table.",
          "unique_fl": true,
          "primary_key": true,
          "default_value": null,
          "col_type": "Unique_Identifier"
        }
      },
      {
        "name": "username",
        "type": "String",
        "nullable": false,
        "metadata": {
          "description": "Unique username for login to the system. Used as the primary identifier for user authentication.",
          "unique_fl": true,
          "default_value": null,
          "col_type": "Unique_Identifier"
        }
      },
      {
        "name": "email",
        "type": "String",
        "nullable": false,
        "metadata": {
          "description": "unique email address for 